# Course Creation Example

This notebook demonstrates how to use the `CourseCreator` to generate course ideas and outlines.

## Import and instantiate the CourseCreator class

Initialize a `CourseCreator` instance. All course creation is performed through this object.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()

# Check the key
key = os.getenv("OPENAI_API_KEY")
print(f"Key: {key[:8]}...{key[-4:]}")
# Test a simple API call
client = OpenAI()
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # cheapest model
        messages=[{"role": "user", "content": "Say hello"}],
        max_tokens=5
    )
    print(f"Success! Response: {response.choices[0].message.content}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
from openai import OpenAI
client = OpenAI()

models_to_test = ["gpt-4o", "o3-mini", "gpt-4.1"]

for model in models_to_test:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Hi"}],
            #max_tokens=5
        )
        print(f"✓ {model}: works")
    except Exception as e:
        print(f"✗ {model}: {e}")

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

key = os.getenv("GOOGLE_API_KEY")
if key:
    print(f"Key: {key[:8]}...{key[-4:]}")

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=key)
    response = llm.invoke("Say hello")
    print(f"Success! Response: {response.content}")
else:
    print("No GOOGLE_API_KEY found in .env")
    print("Get one free at: https://aistudio.google.com/apikey")

In [1]:
from denario.course import CourseCreator

creator = CourseCreator(project_dir="./course_project", clear_project_dir=False)

Run directory: ./course_project/20251126_165929


In [3]:
import denario.prompts.course_outline as outline_prompts
import denario.prompts.course_idea as idea_prompts

print("--- Course Idea Prompt ---")
print(idea_prompts.course_idea_planner_prompt)
print("\n--- Course Outline Prompt ---")
print(outline_prompts.course_outline_researcher_prompt)

--- Course Idea Prompt ---

Given the topic of interest, the duration of the course, and the audience, make a plan according to the following instructions: 
- Ask idea_maker to generate 5 new course ideas related to the topic of interest.
- Ask idea_hater to critique these ideas for pedagogy, feasibility and audience fit.
- Ask idea_maker to select and improve 2 out of the 5 course ideas given the output of the idea_hater.
- Ask idea_hater to critique the 2 improved ideas. 
- Ask idea_maker to select the best idea out of the 2. 
- Ask idea_maker to report the best idea in the form of a course title with a 5-sentence description. 

The goal of this task is to generate a course idea based on the topic of interest. 
IMPORTANT: The course idea must go beyond generic benefits. It should prioritize **technical depth**, **specific tools**, and **actionable frameworks**.
Don't suggest to perform any calculations or analyses here. The only goal of this task is to obtain the best possible course

## Set the course topic, audience, and duration

Specify the subject you want to teach, who the course is for, and how long it will be.

In [ ]:
creator.set_topic(
    topic="ChatGPT Professional Proficiency: From Basics to Advanced Features",
    audience="Professionals in different fields seeking deep proficiency without coding. They want to go from basics in LLMs such as prompt and context to medium-level features. They want to explore a large set of tools currently offered by ChatGPT. ",
    duration="24 hours"
)

creator.show_topic()


Topic: ChatGPT Professional Proficiency: From Basics to Advanced Features
Target Audience: Professionals in different fields seeking deep proficiency without coding. They want to go from basics in LLMs such as prompt and context to medium-level features. They want to explore a large set of tools currently offered by ChatGPT. 
Duration: 8 hours


## Generate a course idea

Use the `idea_maker` and `idea_hater` agents to brainstorm and refine course ideas.

In [5]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
creator.generate_idea()
#creator.generate_idea(
#    idea_maker_model="gpt-4o-mini",
#    idea_hater_model="gpt-4o-mini",
#    planner_model="gpt-4o-mini",
#    plan_reviewer_model="gpt-4o-mini",
#    orchestration_model="gpt-4o-mini",
#    formatter_model="gpt-4o-mini",
#)
# Original models (require higher OpenAI tier):
# creator.generate_idea(
#     idea_maker_model="gpt-4o",
#     idea_hater_model="o3-mini",
#     planner_model="gpt-4o",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_idea()

Generating course idea...
Created context directory:  /Users/elenahernandez/projects/agents/Denario/examples/course_project/20251126_165929/idea_generation_output/context

Topic: ChatGPT Professional Proficiency: From Basics to Advanced Features
Target Audience: Professionals in different fields seeking deep proficiency without coding. They want to go from basics in LLMs such as prompt and context to medium-level features. They want to explore a large set of tools currently offered by ChatGPT. 
Duration: 8 hours


--------------------------------------------------------------------------------
Setting agents for the session...

             Model       agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 plan_setter 0.00118            536                 13           549

--------------------------------------------------------------------------------
Plan constraints have been logged.

------------------------------------------------------------------------


Course Idea:
	* Mastering ChatGPT: From Prompt Crafting to Contextual Mastery
		- This course offers a structured progression from foundational skills in prompt engineering to advanced techniques like context layering.
		- Participants will engage in an 8-hour roadmap with explicit time blocks, ensuring a comprehensive learning experience.
		- The course includes hands-on exercises with specific ChatGPT tools, focusing on real-world applications.
		- Detailed exercises on advanced topics enhance technical depth, providing participants with actionable skills.
		- The course suggests improvements with specific examples using ChatGPT functionalities and feedback loops for prompt evaluation, ensuring practical utility and engagement.

        

## Generate a course outline

Use the `researcher` agent to create a detailed course outline with modules, learning goals, and takeaways.

In [6]:
# Using gpt-4o-mini (available on lower OpenAI tiers)
creator.generate_outline()
#creator.generate_outline(
#    researcher_model="gpt-4o-mini",
#    planner_model="gpt-4o-mini",
#    plan_reviewer_model="gpt-4o-mini",
#    orchestration_model="gpt-4o-mini",
#    formatter_model="gpt-4o-mini", 
#)
# Original models (require higher OpenAI tier):
# creator.generate_outline(
#     researcher_model="gpt-4.1-2025-04-14",
#     planner_model="gpt-4.1-2025-04-14",
#     plan_reviewer_model="o3-mini",
#     orchestration_model="gpt-4.1",
#     formatter_model="o3-mini",
# )
creator.show_outline()

Generating course outline...
Created context directory:  /Users/elenahernandez/projects/agents/Denario/examples/course_project/20251126_165929/course_outline_generation_output/context
Generate the course outline based on the course idea provided above.

--------------------------------------------------------------------------------
Setting agents for the session...

             Model       agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 plan_setter 0.00136            650                  8           658

--------------------------------------------------------------------------------
Plan constraints have been logged.

--------------------------------------------------------------------------------

Calling planner...

             Model   agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-2025-04-14 planner 0.00527            865                443          1308
**Plan:**

- Step 1:
    * sub-task: Analyze the course idea and ident

# Mastering ChatGPT: From Prompt Crafting to Contextual Mastery

### Overview:
This one-day, hands-on course is designed for professionals, educators, and knowledge workers seeking to leverage ChatGPT for real-world problem-solving and productivity. No technical background is required. Across 10 focused modules, participants will progress from foundational prompt engineering to advanced context management, using explicit tools and features of ChatGPT. The course emphasizes practical exercises, actionable feedback, and iterative improvement for immediate workplace application.

---

## 1. Introduction to ChatGPT and Generative AI
### Description:
This foundational module introduces the core concepts of generative AI, focusing on ChatGPT’s architecture, capabilities, and limitations. It orients participants to the ChatGPT interface and highlights key features, setting the stage for effective and responsible use.
### Takeaways:
- Clear understanding of what ChatGPT is and how it works
- Awareness of generative AI’s strengths and boundaries
- Familiarity with the ChatGPT user interface and core features
### Learning Goals:
- Participants describe the architecture and function of ChatGPT.
- Participants identify appropriate and inappropriate use cases.
- Participants navigate the ChatGPT interface and locate key features.
### Exercise Description:
Duration: 15 minutes, Debrief: 5 minutes

---

## 2. Foundations of Prompt Engineering
### Description:
This session covers the principles of prompt engineering, focusing on how prompt structure, clarity, and specificity influence ChatGPT’s outputs. Participants will experiment with basic prompt formats and observe their effects.
### Takeaways:
- Understanding of prompt structure and its impact
- Ability to craft clear, goal-oriented prompts
- Recognition of common prompt pitfalls
### Learning Goals:
- Participants construct effective basic prompts for diverse tasks.
- Participants analyze how prompt wording changes model responses.
- Participants identify and correct ambiguous or ineffective prompts.
### Exercise Description:
Duration: 20 minutes, Debrief: 10 minutes

---

## 3. Exploring ChatGPT Tools and Features
### Description:
Participants will gain hands-on experience with specific ChatGPT tools, including the Code Interpreter (Advanced Data Analysis), File Uploads, Custom Instructions, and the use of GPTs (custom bots). The session emphasizes practical navigation and feature selection.
### Takeaways:
- Familiarity with ChatGPT’s toolset and their applications
- Ability to select appropriate tools for specific tasks
- Confidence in using advanced features like file uploads and custom instructions
### Learning Goals:
- Participants utilize the Code Interpreter for data-related tasks.
- Participants upload and process files within ChatGPT.
- Participants configure and apply Custom Instructions for tailored outputs.
### Exercise Description:
Duration: 25 minutes, Debrief: 10 minutes

---

## 4. Iterative Prompt Refinement and Feedback Loops
### Description:
This module introduces iterative prompt development, emphasizing the use of feedback loops to improve output quality. Participants will practice evaluating and refining prompts using ChatGPT’s conversation history and output review.
### Takeaways:
- Mastery of iterative prompt refinement
- Skills in using feedback to enhance prompt effectiveness
- Techniques for leveraging conversation history for improvement
### Learning Goals:
- Participants evaluate ChatGPT outputs for accuracy and relevance.
- Participants iteratively refine prompts based on model feedback.
- Participants document and apply lessons learned from prompt iterations.
### Exercise Description:
Duration: 20 minutes, Debrief: 10 minutes

---

## 5. Context Layering and Memory Management
### Description:
Participants will learn advanced techniques for managing context, including context layering, summarization, and the use of persistent memory features (where available). The session addresses strategies for maintaining coherence in multi-turn conversations.
### Takeaways:
- Understanding of context layering and its importance
- Ability to manage and summarize conversation context
- Strategies for leveraging memory features in ChatGPT
### Learning Goals:
- Participants apply context layering to complex tasks.
- Participants summarize and reintroduce context for continuity.
- Participants utilize memory features to personalize interactions.
### Exercise Description:
Duration: 25 minutes, Debrief: 10 minutes

---

## 6. Advanced Prompt Engineering: Role Assignment and System Prompts
### Description:
This module explores advanced prompt engineering techniques, including role assignment, system prompts, and persona creation. Participants will design prompts that guide ChatGPT’s behavior and tone for specialized applications.
### Takeaways:
- Proficiency in assigning roles and personas to ChatGPT
- Ability to use system prompts for consistent output style
- Skills in customizing ChatGPT’s responses for specific audiences
### Learning Goals:
- Participants create system prompts to control model behavior.
- Participants assign roles and personas for tailored outputs.
- Participants evaluate the impact of advanced prompts on response quality.
### Exercise Description:
Duration: 20 minutes, Debrief: 10 minutes

---

## 7. Real-World Applications and Workflow Integration
### Description:
This session focuses on integrating ChatGPT into daily workflows, including document drafting, data analysis, brainstorming, and research. Participants will explore use cases and best practices for maximizing productivity.
### Takeaways:
- Practical strategies for workflow integration
- Examples of ChatGPT in professional contexts
- Awareness of automation and productivity enhancements
### Learning Goals:
- Participants identify and implement ChatGPT in their workflows.
- Participants automate routine tasks using ChatGPT features.
- Participants evaluate the impact of ChatGPT on productivity.
### Exercise Description:
Duration: 20 minutes, Debrief: 10 minutes

---

## 8. Evaluating and Troubleshooting ChatGPT Outputs
### Description:
Participants will develop skills in critically evaluating ChatGPT outputs, identifying errors, biases, and hallucinations. The session covers troubleshooting strategies and the use of external validation tools.
### Takeaways:
- Ability to detect and address model errors
- Understanding of bias and hallucination risks
- Familiarity with validation and troubleshooting techniques
### Learning Goals:
- Participants assess outputs for factual accuracy and bias.
- Participants apply troubleshooting steps to problematic outputs.
- Participants use external tools for output validation.
### Exercise Description:
Duration: 20 minutes, Debrief: 10 minutes

---

## 9. Custom GPTs and Automation
### Description:
This module introduces the creation and deployment of custom GPTs (custom bots) using the ChatGPT platform. Participants will design, configure, and test custom bots for specialized tasks and automation.
### Takeaways:
- Understanding of custom GPT creation and deployment
- Skills in configuring bots for specific use cases
- Awareness of automation opportunities with custom GPTs
### Learning Goals:
- Participants design and configure a custom GPT for a defined task.
- Participants test and iterate on custom bot performance.
- Participants integrate custom GPTs into their workflow.
### Exercise Description:
Duration: 25 minutes, Debrief: 10 minutes

---

## 10. Capstone: End-to-End Project and Peer Review
### Description:
In the final module, participants will apply all learned skills to a comprehensive, real-world project. They will design, implement, and refine a ChatGPT-powered solution, followed by structured peer review and feedback.
### Takeaways:
- Experience in end-to-end application of ChatGPT skills
- Constructive feedback from peers and instructors
- Confidence in independently deploying ChatGPT solutions
### Learning Goals:
- Participants design and execute a complete ChatGPT project.
- Participants present and justify their approach to peers.
- Participants provide and receive actionable feedback.
### Exercise Description:
Duration: 40 minutes, Debrief: 20 minutes

---

## Further Resources

- OpenAI ChatGPT Documentation: https://platform.openai.com/docs/guides/gpt
- OpenAI Cookbook: https://github.com/openai/openai-cookbook
- Prompt Engineering Guide: https://www.promptingguide.ai/
- ChatGPT Community Forums: https://community.openai.com/
- Selected readings on generative AI ethics and best practices

---

## Alternative: Set idea manually

If you already have a course idea, you can set it directly and skip the idea generation step.

In [ ]:
creator.set_idea(idea=r"""
 Course Title: ChatGPT Premium Unlocked: Tools and Techniques for Business Excellence
 
- This course offers a comprehensive exploration of the advanced tools available in the ChatGPT premium seat, focusing on features like execution flows and custom instructions.
- Participants will engage in interactive workshops and analyze real-world business case studies to enhance practical relevance and applicability.
- Each module includes in the end 1 or 2 practical exercises to apply the tools learned.
- The course inlcudes some ethical AI practices, providing examples directly linked to business implications to ensure responsible usage.
- Scaffolding steps, such as a mini project or simulation exercise, are included to facilitate the transition from tool exploration to practical application.
- Workshop segments are mapped to specific business outcomes, such as increased operational efficiency, to clearly measure the learning impact and ensure actionable insights.

 """)


creator.generate_outline()
creator.show_outline()

## Full workflow in one call

Alternatively, run the entire workflow (idea + outline generation) with a single method call.

In [ ]:
# creator2 = CourseCreator(project_dir="./another_course", clear_project_dir=True)
# creator2.set_topic(
#     topic="Data Visualization with Python",
#     audience="Data analysts familiar with Excel",
#     duration="1 day (4 hours)"
# )
# creator2.run()  # Generates both idea and outline
# creator2.show_outline()